---

# CSCI 3202, Spring 2023
# Final Project
# Project Due: Wednesday, May 3 at 9:00 PM
## Proposals Due: Wednesday, April 12 at 9:00 PM


You have two options for completing your final project for this course. 

#### Option 1 ####
The first option is presented in this notebook and involves implementing a Connect Four game with AB pruning and A* as player strategies. 

#### Option 2 ####
The second option is to design your own project that includes any of the algorithms we've discussed throughout the semester, or an algorithm that you're interested in learning that we haven't discussed in class. Your project also needs to include some kind of analysis of how it performed on a specific problem. If you're interested in the design your own project option, you need to discuss your idea with one of the course instructors to get approval. If you do a project without getting approval, you will receive a 0 regardless of the quality of the project. 

**The rules:**

1. Choose EITHER the given problem to submit OR choose your own project topic. 

2. If you choose your own project topic, please adhere to the following guidelines:
- Send an email to the course instructor before Wednesday, April 12, with a paragraph description of your project. I will respond in 24 hours.
- The project can include an algorithm we've discussed throughout the semester or an algorithm that you're been curious to learn. Please don't recycle a project that you did in another class. 
- If you do your own project without prior approval, you will receive a 0 for this project.
- Your project code, explanation, and results must all be contained in a Jupyter notebook. 

3. All work, code and analysis must be **your own**.
4. You may use your course notes, posted lecture slides, textbook, in-class notebooks and homework solutions as resources.  You may also search online for answers to general knowledge questions, like the form of a probability distribution function, or how to perform a particular operation in Python. You may not use entire segments of code as solutions to any part of this project, e.g. if you find a Python implementation of policy iteration online, you can't use it.
5. You may **not** post to message boards or other online resources asking for help.
6. **You may not collaborate with classmates or anyone else.**
7. This is meant to be like a coding portion of an exam. So, we will be much less helpful than we typically are with homework. For example, we will not check answers, help debug your code, and so on.
8. If you have a question, post it first as a **private** Piazza message. If we decide that it is appropriate for the entire class, then we will make it a public post (and anonymous).
9. If any part of the given project or your personal project is left open-ended, it is because we intend for you to code it up however you want. Our primary concern is with the plots/analysis that your code produces. Feel free to ask clarifying questions though.

Violation of these rules will result in an **F** and a trip to the Honor Code council.

---
**By writing your name below, you agree to abide by these rules:**

**Your name:** Matthew Riopelle

---


---

Some useful packages and libraries:



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
import heapq
import unittest
from scipy import stats
import copy as cp
from time import time

---

## Problem 1: Game Theory - Playing "intelligent" Connect Four (100 points total)

Connect Four is a two-player game where the objective is to get four pieces in a row - horizontally, vertically, or diagonally. Check out this video if you're unfamiliar with the game: https://www.youtube.com/watch?v=utXzIFEVPjA.

### (1a)   Defining the Connect Four class structure (10 points)

We've provided the humble beginnings of a Connect Four game. You need to fill in this class structure for Connect Four using what we did during class as a guide, and then implement min-max search with AB pruning, and A* search with at least one heuristic function. The class structure includes the following:

* `moves` is a list of columns to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`. Note that when a 'move' is made, the column must have an open slot and the piece must drop to the lowest row. 
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` returns the utility of the current state if the player is Red and $-1 \cdot$ utility if the player is Black.
* `display(self)` is a method to display the current game `state`. You get it for free because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if Red wins, 0 if draw, -1 if Black wins). `player1` and `player2` are functional arguments that we will deal with in parts **1b** and **1d**.

Some notes:
* Assume Red always goes first.
* Do **not** hard-code for 6x7 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [2]:
class State:
    def __init__(self, moves):
        self.to_move = 'R'
        self.utility = 0
        self.board = {}
        self.moves = moves

class ConnectFour:
    def __init__(self, nrow, ncol, nwin):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        moves = [(row,col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        self.state = State(moves)
        
    def hueristic(self):
        "a search hueristic which counts the number of checkers on the board"
        count = 0
        for row in range(1, self.nrow+1):
            for col in range(1, self.ncol+1):
                if(self.state.board.get((row,col))):
                    count+=1
        return count

    def result(self, move, state):
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (R or B)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''
        
        if move not in self.actions(state):
            return state
        # Return a copy of the updated state:
        #   compute utility, update the board, remove the move, update whose turn
        new_state = cp.deepcopy(state)
        new_state.utility = self.compute_utility(move, state)
        new_state.board[move] = state.to_move
        new_state.moves.remove(move)
        new_state.to_move = ('B' if state.to_move == 'R' else 'R')
        return new_state
    
    def actions(self, state):
        return [(x, y) for (x, y) in state.moves
                if x == self.nrow or (x + 1 , y ) in state.board]

    def compute_utility(self, move, state):
        '''
        What is the utility of making move `move` in state `state`?
        If 'R' wins with this move, return 1;
        if 'B' wins return -1;
        else return 0.
        
        Got utility function from class github
        '''        
        
        # your code goes here...
        row, col = move
        player = state.to_move
        
        # create a hypothetical copy of the board, with 'move' executed
        board = cp.deepcopy(state.board)
        board[move] = player
        if (self.k_in_row(board, move, player, (0, 1)) or self.k_in_row(board, move, player, (1, 0)) or self.k_in_row(board, move, player, (1, -1)) or self.k_in_row(board, move, player, (1, 1))):
            return +1 if player == 'R' else -1
        else:
            return 0
        
    def k_in_row(self, board, move, player, delta_x_y):
        """Return true if there is a line through move on board for player."""
        (delta_x, delta_y) = delta_x_y
        x, y = move
        n = 0  # n is number of moves in row
        while board.get((x, y)) == player:
            n += 1
            x, y = x + delta_x, y + delta_y
        x, y = move
        while board.get((x, y)) == player:
            n += 1
            x, y = x - delta_x, y - delta_y
        n -= 1  # Because we counted move itself twice
        return n >= self.nwin



    def game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''
        
        # your code goes here...  
        return state.utility!=0 or len(state.moves)==0    

    def utility(self, state, player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''
        
        # your code goes here...
        return state.utility if player=='R' else -state.utility

    def display(self):
        board = self.state.board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()

    def play_game(self, player1, player2):
        '''Play a game of Connect Four!'''
        
        # your code goes here...
        turn_limit = self.nrow*self.ncol  # limit in case of buggy code
        turn = 0
        while turn<=turn_limit:
            for player in [player1, player2]:
                turn += 1
                move = player(self)
                self.state = self.result(move, self.state)
                if self.game_over(self.state):
                    return self.state.utility
                

### (1b) Define a random player (10 points)

Define a function `random_player` that takes a single argument of the `ConnectFour` class and returns a random move out of the available legal moves in the `state` of the `ConnectFour` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [3]:
def random_player(game):
    '''A player that chooses a legal move at random out of all
    available legal moves in ConnectFour state argument'''
    
    # your code goes here...
    possible_actions = game.actions(game.state)
    return possible_actions[np.random.randint(low=0, high=len(possible_actions))]



We know from experience and/or because I'm telling you right now that if two `random_player`s play many games of ConnectFour against one another, whoever goes first will win about 55% of the time.  Verify that this is the case by playing at least 1,000 games between two random players. Report the proportion of the games that the first player has won.**(Chris: is this true for TicTacToe, or Connect Four)**

**"Unit tests":** If you are wondering how close is close enough to 55%, I simulated 100 tournaments of 1,000 games each. The min-max range of win percentage by the first player was 52-59%.

In [4]:
# 1000 games between two random players

# Your code here
niter = 10000
wins = 0
draws = 0
losses = 0
for k in range(niter):
    cn4 = ConnectFour(3,4,3)
    out = cn4.play_game(random_player, random_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
        
print('First-player winning percentage for connect 3 = {}'.format(wins/niter))
print('\n\nFirst-player losing percentage for connect 3 = {}'.format(1-(wins/niter)))
print('Draws for connect 3 = {}'.format(draws/niter))

First-player winning percentage for connect 3 = 0.6236


First-player losing percentage for connect 3 = 0.37639999999999996
Draws for connect 3 = 0.0429


### (1c) What about playing randomly on different-sized boards? (20 points)

What does the long-term win percentage appear to be for the first player in a 10x10 ConnectFour tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **1b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

In [5]:
# 1000 games between two random players

# Your code here
niter = 10000
wins = 0
draws = 0
losses = 0
for k in range(niter):
    cn4 = ConnectFour(10,10,4)
    out = cn4.play_game(random_player, random_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
        
print('First-player winning percentage for connect 3 = {}'.format(wins/niter))
print('\n\nFirst-player losing percentage for connect 3 = {}'.format(1-(wins/niter)))
print('Draws for connect 3 = {}'.format(draws/niter))

First-player winning percentage for connect 3 = 0.558


First-player losing percentage for connect 3 = 0.44199999999999995
Draws for connect 3 = 0.0


### Response to 1b and 1c
As we can observe, there is a pretty drastic decrease in the win percentage for the first player between the two boards. The increase in board size also causes a substantially larger amount of losses for the player. The draws are virtually non existent now. This is certainly due to the fact that there is a significantly larger board, allowing for an exponentially larger set of potential states. The likelihood of filling up all 100 slots before a win by either side is achieved is incredibly low. So in short, the expanded board cause more losses for the first player.

### (1d) Define an alpha-beta player (20 points)

Alright. Let's finally get serious about our Connect Four game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `ConnectFour` class object and returns the minimax move in the `state` of the `ConnectFour` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [6]:
# Your code here
#alpha-beta search alg from class github
ab_count = 0
def alpha_beta_search(game):

    state = game.state
    player = state.to_move

    # Functions used by alpha_beta
    def max_value(state, alpha, beta):
        global ab_count
        ab_count += 1
        if game.game_over(state):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(a,state), alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta):
        global ab_count
        ab_count += 1
        if game.game_over(state):
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(a,state), alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(a,state), best_score, beta)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

def alpha_beta_player(game):
    return alpha_beta_search(game)

Verify that your alpha-beta player code is working appropriately through the following tests, using a standard 6x7 ConnectFour board. Run **10 games for each test**, and track the number of wins, draws and losses. Report these results for each case.

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. Two alpha-beta players should always draw. One player is the max player and the other player is the min player.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. You are welcome to run more than 10 tests if you'd like. 

In [12]:
# Your code here
niter = 100
wins = 0
draws = 0
losses = 0
for k in range(niter):
    cn4 = ConnectFour(3,4,3)
    out = cn4.play_game(alpha_beta_player, random_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
        
print('First-player winning percentage for connect 3 (alpha-beta vs. random) = {}'.format(wins/niter))
print('\n\nFirst-player losing percentage for connect 3 = {}'.format(1-(wins/niter)))
print('Draws for connect 3 = {}'.format(draws/niter))

First-player winning percentage for connect 3 (alpha-beta vs. random) = 1.0


First-player losing percentage for connect 3 = 0.0
Draws for connect 3 = 0.0


### (1e) What has pruning ever done for us? (10 points)

Calculate the number of number of states expanded by the minimax algorithm, **with and without pruning**, to determine the minimax decision from the initial 6x7 ConnectFour board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Then compute the percent savings that you get by using alpha-beta pruning. i.e. Compute $\frac{\text{Number of nodes expanded with pruning}}{\text{Number of nodes expanded with minimax}} $

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

In [8]:
# Your code here.
minmax_count = 0
ab_count = 0
def minmax_decision(game):
    """Given a state in a game, calculate the best move by searching
    forward all the way to the terminal states. [Figure 5.3]"""
    state = game.state
    player = state.to_move

    def max_value(state):
        global minmax_count
        minmax_count += 1
        if game.game_over(state):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(a, state)))
        return v

    def min_value(state):
        global minmax_count
        minmax_count+= 1
        if game.game_over(state):
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(a,state)))
        return v

    # Body of minmax_decision:
    x = 0
    return max(game.actions(state), key=lambda a: min_value(game.result(a, state)))

def minimax_player(game):
    return minmax_decision(game)
cn4 = ConnectFour(3,4,3)
cn4.play_game(alpha_beta_player, random_player)
print("The number of expansions for AB pruning was: {}".format(ab_count))
cn4 = ConnectFour(3,4,3)
cn4.play_game(minimax_player, random_player)
print("The number of expansions for minimax was: {}".format(minmax_count))
print("The ratio of AB nodes searched to minimax searched was: {}".format(ab_count/minmax_count))
print("The actual % savings is: {}".format((1-ab_count/minmax_count) * 100))

The number of expansions for AB pruning was: 1973
The number of expansions for minimax was: 294051
The ratio of AB nodes searched to minimax searched was: 0.0067097204226477716
The actual % savings is: 99.32902795773522


## 1e Answer
Although we learned in class that AB pruning should take off at least 50%, it does significantly more here because of the utility function only allowing values of -1, 0, and 1. Because of this utility function, once AB has a value of 1 for a subtree (a win), it prunes all other subtrees in the game tree. As checked with Prof Dykes, this amount of savings is to be expected because of this philosophy. So, in this case, AB pruning is faster by more than 99%. 

### (2) A* Search

In Part II of this project, you need to implement a player strategy to employ A* Search in order to win at ConnectFour. To test your A* player, play 10 games against the random player and 10 games against the AB pruning player. 

Write an explanation of this strategy's implementation and performance in comparison to the random player and the AB Pruning player from **1d**.

A lot of the code that you wrote for the minimax player and the Connect Four game structure can be reused for the A* player. However, you will need to write a new utility function for A* that considers the path cost and heuristic cost.


### (2a) Define a heuristic function (20 points)
Your A* player will need to use a heuristic function. You have two options for heurstics: research an existing heuristic for Connect Four, or games similar to Connect Four, and use that. Or, design your own heuristic. Write a one-paragraph description of the heuristic you're using, including a citation if you used an existing heuristic.

For my hueristic, I chose an idea that Prof. Dykes pitched in a piazza post, the link will be at the bottom of this textbox. I would like to have my path cost fixed at a constant for all nodes. This makes sure that my hueristic brings the A* search to the bottom of the tree first. When it arrives, if the utility is a 1, meaning the player will win, it does not have to look at any other parts of the tree. It essentially will examine terminal states, until a preferable utility is achieved. Since my path cost will be fixed for all nodes, I will choose a hueristic that increases as we get closer to the solution. This is because I would like for my path cost + hueristic cost to increase as I get closer to the goal state (3 in a row for Connect3). So, my hueristic will be the number of checkers on the board at a given state. This certainly will not be an amazing hueristic, but I wanted to compare an A* search with a poor search hueristic to these other players to see the outcome.

https://piazza.com/class/lcqhcg50cjm1cf/post/128

**My hueristic function is defined in the connect4 class from above**

### (2b) Compare A* to other algorithms (10 points)
Next, play 10 games of Connect Four using your A* player and a random player and 10 games against the AB pruning player. In four or five paragraphs, report on the outcome. Did one player win more than the other? How often was the game a draw? How many moves did each player make? Were there situations where one player appeared to do better than the other? Given the outcome, are there other heuristics you would like to implement?

In [9]:
# Your code here.
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0, hueristic = 0):
        """Create a search tree Node, derived from a parent by an action."""
        "found this class structure in the class github, made some modifications"
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.hueristic = hueristic
    def __lt__(self, node):
        return self.path_cost + self.hueristic < node.path_cost + node.hueristic

def astar_search(game):
    "the following code has been adapted from prev hw, and class github"
    init_node = Node(game.state, path_cost = 0, hueristic = game.hueristic())
    frontier = [init_node]
    visited = set()
    while frontier:
        "adds node to frontier"
        node = heapq.heappop(frontier)
        "if the game is over, return the state"
        if game.game_over(node.state):
            return node.action
        visited.add(node.state)
        for action in game.actions(node.state):
            child = game.result(action, node.state)
            child_node = Node(child, node, action, node.path_cost + 1, game.hueristic())
            if child_node not in visited and child_node not in frontier:
                heapq.heappush(frontier, child_node)
    return None

def astar_player(game):
    result = astar_search(game)
    if result is None:
        return None 
    else: return result

In [10]:
niter = 100
wins = 0
draws = 0
losses = 0
for k in range(niter):
    cn4 = ConnectFour(3,4,3)
    out = cn4.play_game(astar_player, random_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
        
print('First-player winning percentage for connect 3 (astar vs random) = {}'.format(wins/niter))
print('\n\nFirst-player losing percentage for connect 3 = {}'.format(1-(wins/niter)))
print('Draws for connect 3 = {}'.format(draws/niter))

First-player winning percentage for connect 3 (astar vs random) = 0.85


First-player losing percentage for connect 3 = 0.15000000000000002
Draws for connect 3 = 0.05


In [11]:
niter = 10
wins = 0
draws = 0
losses = 0
for k in range(niter):
    cn4 = ConnectFour(3,4,3)
    out = cn4.play_game(astar_player, alpha_beta_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
        
print('First-player winning percentage for connect 3 (astar vs alphabeta) = {}'.format(wins/niter))
print('\n\nFirst-player losing percentage for connect 3 = {}'.format(1-(wins/niter)))
print('Draws for connect 3 = {}'.format(draws/niter))

First-player winning percentage for connect 3 (astar vs alphabeta) = 0.0


First-player losing percentage for connect 3 = 1.0
Draws for connect 3 = 0.0


### (2b) Writeup
I decided to go ahead and run 100 games for each of these matchups. When my astar algorithm played against a random player, it won roughly 82% of the time. However, when matched up against the alpha beta player, astar had no chance, winning 0% of the games on average.

In terms of draws, there were also no draws in the alpha beta game. However, it seems as though when put against a random player, my astar algorithm draws around 5% of the time.

These results make a lot of sense to me. In terms of the astar vs random player, astar should certainly win the majority of the time. This is because it is a deterministic algorithm, and should therefore be able to find a way to win in most states against a completely stochastic player. Furthermore, the draw makes sense because my hueristic was incredibly weak. Because of this weak hueristic, it makes sense that a random player might win or draw the game every once in a while. In terms of the astar vs alpha beta player, these results do not surprise me either. It is two determinstic algorithms battling it out. However, as stated before, the search hueristic is not a strong one for astar. When put up against a deterministic algorithm that is much stronger than it like alpha beta, it makes a lot of sense why it is never able to win. These two determinstic algorithms also output the same terminal state on every iteration.

Given this outcome, I would really like to add a different, stronger search hueristic. One that I was potentially thinking of is, how many potential game winning moves are available on the board. I do believe that this would be a stronger hueristic to base my findings off of. If I were to employ this different hueristic, I would have to believe that my astar would never lose to a random player, and that it may win some games against the alpha_beta player (although since they are both deterministic I might expect the result to be the same each time). To test this, I would probably want to alter the starting positions of each player to see if the outcomes changed.